In [1]:
import keras
from keras.models import load_model
deepsea_model = load_model("/srv/scratch/avanti/ExPecto/resources/deepseabeluga_keras.h5")

Using TensorFlow backend.
W0822 16:31:40.468896 140525122139904 deprecation_wrapper.py:119] From /users/avanti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0822 16:31:40.496711 140525122139904 deprecation_wrapper.py:119] From /users/avanti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0822 16:31:40.567846 140525122139904 deprecation_wrapper.py:119] From /users/avanti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0822 16:31:40.568900 140525122139904 deprecation_wrapper.py:119] From /users/avanti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Pl

In [7]:
%%bash

#copy over the new K562 dnase file
cp /oak/stanford/groups/akundaje/projects/atlas/dnase_processed/k562_dnase/hg38/cromwell-executions/atac/41bb41c3-c39b-4145-9305-58cc692ef2e4/call-reproducibility_overlap/execution/optimal_peak.narrowPeak.gz hg38_new_k562.narrowPeak.gz

#prep universal dnase negatives - for performance assessment
#universal_negatives=/srv/scratch/annashch/deeplearning/gecco/encode_dnase/all_dnase_idr_peaks.sorted.bed
#merged_universal_neg="merged_universal_neg.bed.gz"
#cat $universal_negatives | mergeBed | gzip -c > $merged_universal_neg
#bedtools intersect -sorted -a $merged_universal_neg -b $universal_negatives -wa -wb | gzip -c > merged_universal_neg_intersected.bed.gz
#./take_best_peak.py | gzip -c > merged_universal_neg_representative_peaks.bed.gz

#Map the representative DNase negatives to hg38 using liftover
module load ucsc_tools
#download the liftover chain file if it doesn't exist already
[[ -e hg19ToHg38.over.chain.gz ]] || wget http://hgdownload.cse.ucsc.edu/goldenpath/hg19/liftOver/hg19ToHg38.over.chain.gz
#run the liftover command
liftOver merged_universal_neg_representative_peaks.bed.gz hg19ToHg38.over.chain.gz hg38_merged_universal_neg_representative_peaks.bed unmapped_merged_universal_neg_representative_peaks.bed
#compress and clean up
gzip hg38_merged_universal_neg_representative_peaks.bed
rm unmapped_merged_universal_neg_representative_peaks.bed
rm merged_universal_neg_representative_peaks.bed.gz

#take 2000bp around the summit of the k562 peaks
zcat hg38_new_k562.narrowPeak.gz | perl -lane 'print $F[0]."\t".(($F[1]+$F[9]))."\t".(($F[1]+$F[9]))' | bedtools slop -g /mnt/data/annotations/by_organism/human/hg20.GRCh38/hg38.chrom.sizes -b 1000 | perl -lane 'if ($F[2]-$F[1]==2000) {print $_}' | gzip -c > 2000bp_around_hg38_new_k562.narrowPeak.gz
#take 2000bp around the summit of the DNAse peaks
zcat hg38_merged_universal_neg_representative_peaks.bed.gz | perl -lane 'print $F[0]."\t".(($F[1]+$F[9]))."\t".(($F[1]+$F[9]))' | bedtools slop -g /mnt/data/annotations/by_organism/human/hg20.GRCh38/hg38.chrom.sizes -b 1000 | perl -lane 'if ($F[2]-$F[1]==2000) {print $_}' | gzip -c > 2000bp_around_hg38_merged_universal_neg_representative_peaks.bed.gz

#filter out any universal dnase that overlap anything in the k562 set 
intersectBed -v -a 2000bp_around_hg38_merged_universal_neg_representative_peaks.bed.gz -b 2000bp_around_hg38_new_k562.narrowPeak.gz -wa | gzip -c > nok562_2000bp_around_hg38_merged_universal_neg_representative_peaks.bed.gz
#clean up the intermediate file
rm 2000bp_around_hg38_merged_universal_neg_representative_peaks.bed.gz

#get the fasta sequences underlying the 2000bp regions
bedtools getfasta -fi /mnt/data/annotations/by_organism/human/hg20.GRCh38/GRCh38.genome.fa -bed 2000bp_around_hg38_new_k562.narrowPeak.gz > 2000bp_around_hg38_new_k562.fa

bedtools getfasta -fi /mnt/data/annotations/by_organism/human/hg20.GRCh38/GRCh38.genome.fa -bed nok562_2000bp_around_hg38_merged_universal_neg_representative_peaks.bed.gz > nok562_2000bp_around_hg38_merged_universal_neg_representative_peaks.fa


bash: line 13: module: command not found
bash: line 17: liftOver: command not found
gzip: hg38_merged_universal_neg_representative_peaks.bed: No such file or directory
rm: cannot remove 'unmapped_merged_universal_neg_representative_peaks.bed': No such file or directory
rm: cannot remove 'merged_universal_neg_representative_peaks.bed.gz': No such file or directory
Error: The requested genome file (/mnt/data/annotations/by_organism/human/hg20.GRCh38/hg38.chrom.sizes) could not be opened. Exiting!
gzip: hg38_merged_universal_neg_representative_peaks.bed.gz: No such file or directory
Error: The requested genome file (/mnt/data/annotations/by_organism/human/hg20.GRCh38/hg38.chrom.sizes) could not be opened. Exiting!
Error: The requested fasta database file (/mnt/data/annotations/by_organism/human/hg20.GRCh38/GRCh38.genome.fa) could not be opened. Exiting!
Error: The requested fasta database file (/mnt/data/annotations/by_organism/human/hg20.GRCh38/GRCh38.genome.fa) could not be opened. Exit

CalledProcessError: Command 'b'\n#copy over the new K562 dnase file\ncp /oak/stanford/groups/akundaje/projects/atlas/dnase_processed/k562_dnase/hg38/cromwell-executions/atac/41bb41c3-c39b-4145-9305-58cc692ef2e4/call-reproducibility_overlap/execution/optimal_peak.narrowPeak.gz hg38_new_k562.narrowPeak.gz\n\n#prep universal dnase negatives - for performance assessment\n#universal_negatives=/srv/scratch/annashch/deeplearning/gecco/encode_dnase/all_dnase_idr_peaks.sorted.bed\n#merged_universal_neg="merged_universal_neg.bed.gz"\n#cat $universal_negatives | mergeBed | gzip -c > $merged_universal_neg\n#bedtools intersect -sorted -a $merged_universal_neg -b $universal_negatives -wa -wb | gzip -c > merged_universal_neg_intersected.bed.gz\n#./take_best_peak.py | gzip -c > merged_universal_neg_representative_peaks.bed.gz\n\n#Map the representative DNase negatives to hg38 using liftover\nmodule load ucsc_tools\n#download the liftover chain file if it doesn\'t exist already\n[[ -e hg19ToHg38.over.c